In [1]:
import pdb
import imp
import os
from pathlib import Path
import torch
import numpy as np
from matplotlib import pyplot
import rasterio
import fastai
from multispectral import corine
from multispectral import coords
from multispectral import windows
from multispectral import tools
from multispectral import bands
import neptune
import infra
import zoo

neptune.init('denised/landcover')
windows_list = os.environ['LANDSAT_DIR'] + '/all_windows.csv'
infra.set_defaults(
    corine_directory=os.environ['CORINE_DIR'],
    traintracker_store=infra.TrainTrackerWebHook(os.environ['TRAINTRACKER_URI']))

In [2]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [15.0,10.0]

In [7]:
allwindows = windows.from_file(windows_list)
(tr_list, val_list) = windows.chunked_split(allwindows)
print(len(tr_list), len(val_list))

In [10]:
infra.set_defaults(silent=True, batch_size=4)

In [12]:
# Some optional callbacks.
tracer = infra.LearnerTracer    # check for Nan or zero in gradients
ender = infra.TrainEnder()        # end after train_n iterations
infra.set_defaults(train_end=40)

# CycleHandler will run metrics more often than once per epoch.  You list the metrics *and* how you 
# wan to handle them (such as CSVLogger or SendToNeptune).
# One detail that is a bit confusing: you have pass a LearnerCallback *class* (like DiceMetric),
# but instantiate a regular Callback *object* (like CSVLogger).  See info for CycleHandler class
# for more details, and infra/callbacks.py for examples
monitor = infra.CycleHandler.Create(n=20,   # how frequently to run
   [ infra.DiceMetric, fastai.metrics.accuracy_thresh, infra.CSVLogger() ])

#learner = zoo.Simple.create(tr_list,val_list, [6,20,30,11], callbacks=[ender], callback_fns=[monitor,tracer])
#learner = zoo.ImageUResNet.create(tr_list,val_list, callback_fns=[tracer])
learner = zoo.MultiUResNet.create(tr_list,val_list, callback_fns=[monitor])

In [13]:
#learner.fit_one_cycle(1)
learner.fit(1, description="testing",neptune=False)

There are many possible ways to look at the model performance; I haven't done much work on this yet.

Here we use`get_prediction_set` to get the learner-predicted outcomes for a specific set of windows that you provide,
and then `collapsepreds` to collapse the multi-label outcome into a single most-likely label, which makes it easier
to look at when plotted.

In [9]:
predset = tools.get_prediction_set(learner, tr_list[:20])
collapsedpreds = tools.collapse_predictions(predset)

`show_predictions` will show four columns: the RGB portion of the original image, the collapsed predicted 
landuse, the collapsed target landuse, and a plot showing where these agree or disagree.

In [11]:
tools.show_predictions(collapsedpreds)